This notebook contains examples of using model index and OPC UA functions to download inverters, strings set and trackers data of the first site from the model index and OPC UA api servers. The aggregated historical data downloaded in the data folder and cell execution time can be noted for each request.

### Import Libraries

In [1]:
# Import the required packeages
import os
import datetime
from dotenv import load_dotenv 
from pathlib import Path

### Import Scripts

In [2]:
from pyprediktormapclient.opc_ua import OPC_UA
from pyprediktormapclient.model_index import ModelIndex
from pyprediktormapclient.auth_client import AUTH_CLIENT
from pyprediktormapclient.analytics_helper import AnalyticsHelper
from pyprediktormapclient.shared import *

In [ ]:
# Consider obtaining the envrionment variables from .env file if you are running this locally from source.
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

In [4]:
username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]
opcua_rest_url = os.environ["OPC_UA_REST_URL"]
opcua_server_url = os.environ["OPC_UA_SERVER_URL"]
model_index_url = os.environ["MODEL_INDEX_URL"]
ory_url = os.environ["ORY_URL"]

In [5]:
# Model index API
model = ModelIndex(url=model_index_url)

### OPC UA server parameters

In [ ]:
model.get_objects_of_type("InverterType")

In [ ]:
# All the sites on the OPC server
sites_json = model.get_objects_of_type("SiteType")
sites = AnalyticsHelper(sites_json)
sites.dataframe

In [ ]:
# Selecting the second site
all_site_ids = sites.list_of_ids()
first_site_id = all_site_ids[1]
first_site_id

In [9]:
# Initate the OPC UA connection
tsdata = OPC_UA(rest_url=opcua_rest_url, opcua_url=opcua_server_url, namespaces=sites.namespaces_as_list(model.get_namespace_array()))

INVERTERS DATA

In [ ]:
# All the inverters on the site
inverters_json = model.get_object_descendants("InverterType", all_site_ids, "PV_Assets")
inverters = AnalyticsHelper(inverters_json)
inverters.dataframe

In [11]:
# List of unique inverter variables
inv_var_list = [
    "ACActivePower",
    "ACCurrentAverage",
    "ACCurrentPhaseA",
    "ACCurrentPhaseB",
    "ACCurrentPhaseC",
    "ACDailyActiveEnergy",
    "ACReactivePower",
    "ACTotalActiveEnergy",
    "ACVoltagePhaseAB",
    "ACVoltagePhaseAC",
    "ACVoltagePhaseBC",
    "DCCurrent",
    "DCPower",
    "DCVoltage",
    "Frequency",
    "PowerFactor",
    "Status",
]

In [ ]:
# Live value inverters data
inv_liv_values = tsdata.get_values(
    inverters.variables_as_list(inv_var_list)
)
inv_liv_values

In [ ]:
# 1 day aggregated historical inverter data
one_day_historic_inverter_data = tsdata.get_historical_aggregated_values(
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(29)),
    pro_interval=60*1000,
    agg_name="Average",
    variable_list=inverters.variables_as_list(inv_var_list),
)
one_day_historic_inverter_data